In [ ]:
# 1. Install Dependencies
!pip install optuna librosa pyyaml pandas matplotlib torchaudio

In [ ]:
# 1. Setup Environment
!git clone https://github.com/Quarkisinproton/IndianBatsModel.git
!pip install optuna librosa pyyaml pandas matplotlib torchaudio

In [ ]:
# 2. Import Modules
import sys
import os
from pathlib import Path

REPO_DIR = '/kaggle/working/IndianBatsModel'
os.chdir(REPO_DIR)

if REPO_DIR not in sys.path:
    sys.path.insert(0, REPO_DIR)

try:
    from MainShitz.data_prep.generate_annotations import generate_annotations
    from MainShitz.data_prep.wombat_to_spectrograms import process_all as generate_spectrograms
    from MainShitz.data_prep.extract_end_frequency import process_all_and_write_csv as extract_features
    from MainShitz.data_prep.whombat_project_to_wombat import convert_whombat_project_to_wombat_jsons
    from MainShitz.train import train_model
    print("Imports successful!")
except ImportError as e:
    print(f"Import Error: {e}")
    print("Please ensure the repository is cloned correctly.")

In [ ]:
# 3. Configuration
import glob
import json
import librosa

WORK_DIR = '/kaggle/working'

# ============================================
# INPUT PATHS - UPDATE THESE FOR YOUR DATASET
# ============================================

# Folders containing bat .wav files
RAW_AUDIO_DIRS = [
    '/kaggle/input/pip-ceylonicusbat-species',
    '/kaggle/input/pip-tenuisbat-species',
]

# Whombat project JSON exports (for bat annotations)
WHOMBAT_PROJECT_JSONS = [
    '/kaggle/input/annotations-tenuis-ceylonicus/tenuis annotations.json',
    '/kaggle/input/annotations-tenuis-ceylonicus/Pip ceylonicus.json',
]

# Noise audio folder (set to None or empty string if you don't have noise data)
NOISE_AUDIO_DIR = '/kaggle/input/noice-files/Noise'

# ============================================
# OUTPUT PATHS (auto-generated in /kaggle/working)
# ============================================
JSON_DIR = os.path.join(WORK_DIR, 'data/annotations_json_folder')
SPECT_OUT = os.path.join(WORK_DIR, 'data/processed/spectrograms')
FEATURES_OUT = os.path.join(WORK_DIR, 'data/processed/features')
FEATURES_CSV = os.path.join(FEATURES_OUT, 'end_frequencies.csv')
MODEL_SAVE_PATH = os.path.join(WORK_DIR, 'models', 'bat_tuned_best.pth')

# Create directories
Path(JSON_DIR).mkdir(parents=True, exist_ok=True)
Path(FEATURES_OUT).mkdir(parents=True, exist_ok=True)
Path(os.path.dirname(MODEL_SAVE_PATH)).mkdir(parents=True, exist_ok=True)

# Add noise dir to audio dirs if it exists
ALL_AUDIO_DIRS = RAW_AUDIO_DIRS.copy()
if NOISE_AUDIO_DIR and os.path.exists(NOISE_AUDIO_DIR):
    ALL_AUDIO_DIRS.append(NOISE_AUDIO_DIR)

print("Configuration set.")
print(f"Audio dirs: {ALL_AUDIO_DIRS}")

In [ ]:
# 4. Convert Annotations (Bats + Noise)
print("="*50)
print("STEP 1: Converting Bat Annotations...")
print("="*50)

for pj in WHOMBAT_PROJECT_JSONS:
    if os.path.exists(pj):
        summary = convert_whombat_project_to_wombat_jsons(
            project_json_path=pj,
            output_dir=JSON_DIR,
            tag_key='Species',
            skip_unlabeled=True,
        )
        print(f"  {os.path.basename(pj)}: {summary.jsons_written} files, {summary.sound_events_written} events")
    else:
        print(f"  WARNING: File not found: {pj}")

print("\n" + "="*50)
print("STEP 2: Generating Noise Annotations...")
print("="*50)

if NOISE_AUDIO_DIR and os.path.exists(NOISE_AUDIO_DIR):
    noise_files = glob.glob(os.path.join(NOISE_AUDIO_DIR, "*.wav"))
    print(f"  Found {len(noise_files)} noise files.")
    
    for nf in noise_files:
        try:
            try:
                dur = librosa.get_duration(path=nf)
            except TypeError:
                dur = librosa.get_duration(filename=nf)
            
            fname = os.path.basename(nf)
            # Create annotation in the same format as wombat converter
            entry = {
                "recording": fname,
                "annotations": [{
                    "start_time": 0.0,
                    "end_time": dur,
                    "label": "Noise"
                }]
            }
            json_name = os.path.splitext(fname)[0] + ".json"
            with open(os.path.join(JSON_DIR, json_name), 'w') as f:
                json.dump(entry, f, indent=2)
        except Exception as e:
            print(f"  Error processing {fname}: {e}")
    print(f"  Generated {len(noise_files)} noise annotation files.")
else:
    print("  No noise directory configured or found. Skipping.")

print("\nAnnotation conversion complete.")

In [ ]:
# 5. Generate Spectrograms
print("="*50)
print("STEP 3: Generating Spectrograms...")
print("="*50)

generate_spectrograms(
    raw_audio_dirs=ALL_AUDIO_DIRS,
    json_dir=JSON_DIR,
    out_dir=SPECT_OUT,
    species_key='label'
)

# Verify output
print("\n--- Verification ---")
if os.path.exists(SPECT_OUT):
    subdirs = [d for d in os.listdir(SPECT_OUT) if os.path.isdir(os.path.join(SPECT_OUT, d))]
    print(f"Found {len(subdirs)} class folders: {subdirs}")
    total = sum(len(os.listdir(os.path.join(SPECT_OUT, d))) for d in subdirs)
    print(f"Total spectrograms: {total}")
else:
    print("ERROR: Output directory does not exist!")

print("\nSpectrogram generation complete.")

In [ ]:
# 6. Extract End-Frequency Features (CRITICAL for CNNWithFeatures model)
print("="*50)
print("STEP 4: Extracting End-Frequency Features...")
print("="*50)

extract_features(
    raw_audio_dirs=ALL_AUDIO_DIRS,
    json_dir=JSON_DIR,
    out_csv=FEATURES_CSV,
    species_key='label'
)

print(f"Features saved to {FEATURES_CSV}")

# Preview
import pandas as pd
df = pd.read_csv(FEATURES_CSV)
print(f"\nFeature CSV has {len(df)} rows.")
print(df.head())

In [ ]:
# 7. Create Smart Tuner Script
import torch

# Check GPU availability
print("="*50)
print("GPU Check")
print("="*50)
if torch.cuda.is_available():
    print(f"CUDA available! Found {torch.cuda.device_count()} GPU(s).")
    for i in range(torch.cuda.device_count()):
        print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("WARNING: CUDA not available. Training will be slow on CPU.")

# Infer num_classes
num_classes = len([p for p in Path(SPECT_OUT).iterdir() if p.is_dir()])
print(f"\nDetected {num_classes} classes from {SPECT_OUT}")

# Create the tuner script
tuner_code = f'''
import optuna
import sys
sys.path.insert(0, "{REPO_DIR}")

from MainShitz.train import train_model
from pathlib import Path

# Fixed paths from notebook
SPECT_OUT = "{SPECT_OUT}"
FEATURES_CSV = "{FEATURES_CSV}"
NUM_CLASSES = {num_classes}

def objective(trial):
    # Suggest hyperparameters
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    
    print(f"\\n--- Trial {{trial.number}} ---")
    print(f"Params: lr={{learning_rate:.6f}}, bs={{batch_size}}, wd={{weight_decay:.6f}}")
    
    # Build config (same structure as kaggle_train.ipynb)
    config = {{
        'data': {{
            'train_spectrograms': SPECT_OUT,
            'features_csv': FEATURES_CSV,  # THIS triggers CNNWithFeatures!
            'num_classes': NUM_CLASSES,
        }},
        'train': {{
            'batch_size': batch_size,
            'learning_rate': learning_rate,
            'weight_decay': weight_decay,
            'num_epochs': 5,  # Short epochs for tuning
            'model_save_path': f'models/trial_{{trial.number}}.pth',
            'num_workers': 4,
        }},
    }}
    
    try:
        # Import here to capture the validation loss
        import io
        import sys
        from contextlib import redirect_stdout
        
        # Capture stdout to parse validation loss
        captured = io.StringIO()
        with redirect_stdout(captured):
            train_model(config)
        
        output = captured.getvalue()
        print(output)  # Still show output
        
        # Parse final validation loss
        final_val_loss = None
        for line in output.splitlines():
            if "FINAL_VAL_LOSS:" in line:
                try:
                    final_val_loss = float(line.split("FINAL_VAL_LOSS:")[1].strip())
                except ValueError:
                    pass
        
        if final_val_loss is None:
            # Fallback: try to find last validation loss
            for line in reversed(output.splitlines()):
                if "Val Loss" in line:
                    try:
                        # Extract number after "Val Loss:"
                        parts = line.split("Val Loss:")[1].split()
                        final_val_loss = float(parts[0].strip(","))
                        break
                    except:
                        pass
        
        if final_val_loss is None:
            print("WARNING: Could not parse validation loss.")
            return 999.0
        
        print(f"Trial {{trial.number}} finished with val_loss={{final_val_loss:.4f}}")
        return final_val_loss
        
    except Exception as e:
        print(f"Trial {{trial.number}} failed: {{e}}")
        return 999.0

if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")
    print("Starting Hyperparameter Optimization...")
    print(f"Using CNNWithFeatures (ResNet18 + end-frequency features)")
    print(f"Dataset: {{SPECT_OUT}}")
    print(f"Features: {{FEATURES_CSV}}")
    print(f"Classes: {{NUM_CLASSES}}")
    print()
    
    study.optimize(objective, n_trials=20)
    
    print("\\n" + "="*50)
    print("OPTIMIZATION COMPLETE")
    print("="*50)
    print("Best Hyperparameters:")
    for key, value in study.best_params.items():
        print(f"  {{key}}: {{value}}")
    print(f"Best Validation Loss: {{study.best_value:.4f}}")
    print("="*50)
'''

with open('smart_tuner.py', 'w') as f:
    f.write(tuner_code)

print("\nCreated smart_tuner.py")
print("  - Uses CNNWithFeatures (ResNet18 + features)")
print("  - Optimizes: learning_rate, batch_size, weight_decay")
print("  - Runs 20 trials with 5 epochs each")

In [ ]:
# 8. Run Hyperparameter Optimization
print("="*50)
print("STEP 5: Running Optuna Hyperparameter Optimization")
print("="*50)

!python smart_tuner.py

In [ ]:
# 9. (Optional) Train Final Model with Best Hyperparameters
# After tuning completes, you can train a full model with more epochs
# using the best hyperparameters found above.

# Example:
# best_lr = 0.001  # Copy from tuning results
# best_bs = 16
# best_wd = 0.0001
#
# final_config = {
#     'data': {
#         'train_spectrograms': SPECT_OUT,
#         'features_csv': FEATURES_CSV,
#         'num_classes': num_classes,
#     },
#     'train': {
#         'batch_size': best_bs,
#         'learning_rate': best_lr,
#         'weight_decay': best_wd,
#         'num_epochs': 50,  # Full training
#         'model_save_path': MODEL_SAVE_PATH,
#         'num_workers': 4,
#     },
# }
#
# train_model(final_config)
# print(f"Final model saved to {MODEL_SAVE_PATH}")

In [ ]:
# 2. Clone Repository
!git clone https://github.com/Quarkisinproton/IndianBatsModel.git
%cd IndianBatsModel

In [ ]:
# 3. Patch Codebase (Fixes & Features)

# A. Fix Syntax Error in whombat_project_to_wombat.py
file_path = 'MainShitz/data_prep/whombat_project_to_wombat.py'
try:
    with open(file_path, 'r') as f:
        content = f.read()
    # Fix missing colon if present
    bad_syntax = "if not ann_list continue"
    good_syntax = "if not ann_list: continue"
    if bad_syntax in content:
        content = content.replace(bad_syntax, good_syntax)
        with open(file_path, 'w') as f:
            f.write(content)
        print("Fixed syntax error in whombat_project_to_wombat.py")
except FileNotFoundError:
    print(f"Warning: {file_path} not found.")

# B. Patch train.py to report Final Validation Loss
train_script_path = 'MainShitz/train.py'
try:
    with open(train_script_path, 'r') as f:
        content = f.read()

    if "FINAL_VAL_LOSS" not in content:
        target_str = "print(f\"Training curves saved to {plot_path}\")"
        new_code = """
    print(f"Training curves saved to {plot_path}")

    # Report final validation loss for hyperparameter tuning
    if val_losses:
        print(f"FINAL_VAL_LOSS: {val_losses[-1]}")
"""
        if target_str in content:
            content = content.replace(target_str, new_code)
            with open(train_script_path, 'w') as f:
                f.write(content)
            print("Successfully patched train.py")
        else:
            print("WARNING: Could not find target string to patch train.py.")
    else:
        print("train.py already contains FINAL_VAL_LOSS reporting.")
except FileNotFoundError:
    print(f"Warning: {train_script_path} not found.")

In [ ]:
# 4. Create smart_tuner.py
tuner_code = """
import optuna
import yaml
import os
import subprocess
import sys

def objective(trial):
    # 1. Suggest Hyperparameters
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    
    print(f"\\n--- Trial {trial.number} ---")
    print(f"Params: lr={learning_rate}, bs={batch_size}, wd={weight_decay}")

    # 2. Load Base Config
    base_config_path = 'configs/config.yaml'
    if not os.path.exists(base_config_path):
        raise FileNotFoundError(f"Config file not found: {base_config_path}")
        
    with open(base_config_path, 'r') as f:
        config = yaml.safe_load(f)
    
    if 'train' not in config:
        config['train'] = {}
        
    config['train']['learning_rate'] = learning_rate
    config['train']['batch_size'] = batch_size
    config['train']['weight_decay'] = weight_decay
    
    model_save_path = os.path.join('models', f'trial_{trial.number}.pth')
    config['train']['model_save_path'] = model_save_path
    
    temp_config_path = f'temp_config_{trial.number}.yaml'
    with open(temp_config_path, 'w') as f:
        yaml.dump(config, f)
        
    # 3. Run Training
    cmd = [sys.executable, "-m", "MainShitz.train", "--config", temp_config_path]
    
    try:
        result = subprocess.run(cmd, capture_output=True, text=True, check=True)
        output = result.stdout
        
        # Print GPU info from the training script output
        for line in output.splitlines():
            if "GPU" in line or "device" in line:
                print(f"  [Train Output] {line}")

        final_val_loss = None
        for line in output.splitlines():
            if "FINAL_VAL_LOSS:" in line:
                try:
                    final_val_loss = float(line.split("FINAL_VAL_LOSS:")[1].strip())
                except ValueError:
                    pass
        
        if final_val_loss is None:
            print("Warning: Could not find FINAL_VAL_LOSS in output.")
            return 999.0
            
        return final_val_loss

    except subprocess.CalledProcessError as e:
        print(f"Training failed for trial {trial.number}")
        print("Error:", e.stderr)
        return 999.0
        
    finally:
        if os.path.exists(temp_config_path):
            os.remove(temp_config_path)

if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")
    print("Starting Hyperparameter Optimization...")
    study.optimize(objective, n_trials=20)
    
    print("\\n" + "="*40)
    print("Optimization Complete")
    print("="*40)
    print("Best Hyperparameters:")
    for key, value in study.best_params.items():
        print(f"  {key}: {value}")
    print(f"Best Validation Loss: {study.best_value}")
    print("="*40)
"""

with open('smart_tuner.py', 'w') as f:
    f.write(tuner_code)
print("Created smart_tuner.py")

In [ ]:
import os
import json
import glob
import librosa
import sys

# Ensure we are in the right directory for imports
if os.getcwd().split('/')[-1] != 'IndianBatsModel':
    if os.path.exists('IndianBatsModel'):
        os.chdir('IndianBatsModel')
    sys.path.append('.')

from MainShitz.data_prep.wombat_to_spectrograms import process_all as generate_spectrograms
from MainShitz.data_prep.whombat_project_to_wombat import convert_whombat_project_to_wombat_jsons

# --- CONFIGURATION ---

# 1. Input Paths (Adjust these to match your Kaggle Dataset structure)
#    These are the folders containing your .wav files
RAW_AUDIO_DIRS = [
    '/kaggle/input/annotations-tenuis-ceylonicus/Pip ceylonicus',
    '/kaggle/input/annotations-tenuis-ceylonicus/Pip._tenuis'
]

#    These are the JSON exports from Whombat
WHOMBAT_PROJECT_JSONS = [
    '/kaggle/input/annotations-tenuis-ceylonicus/tenuis annotations.json',
    '/kaggle/input/annotations-tenuis-ceylonicus/Pip ceylonicus.json',
]

#    Path to your noise data (UPDATE THIS if your folder name is different)
NOISE_AUDIO_DIR = '/kaggle/input/noise-data' 

# 2. Output Paths (In the writable /kaggle/working directory)
JSON_DIR = '/kaggle/working/data/annotations_json_folder'
SPECT_OUT = '/kaggle/working/data/processed/spectrograms'

os.makedirs(JSON_DIR, exist_ok=True)
os.makedirs(SPECT_OUT, exist_ok=True)

# --- EXECUTION ---

# 1. Convert Bat Annotations
print("Converting Bat Annotations...")
for pj in WHOMBAT_PROJECT_JSONS:
    # Fix: Only pass 2 arguments (Input File, Output Dir)
    convert_whombat_project_to_wombat_jsons(pj, JSON_DIR)

# 2. Generate Noise Annotations
print("Generating Noise Annotations...")
noise_files = glob.glob(os.path.join(NOISE_AUDIO_DIR, "*.wav"))
print(f"Found {len(noise_files)} noise files.")

noise_annotations = []
for nf in noise_files:
    try:
        # Handle different librosa versions for duration check
        try:
            dur = librosa.get_duration(path=nf)
        except TypeError:
            dur = librosa.get_duration(filename=nf)
            
        # Create a simple annotation for the whole file
        ann = {
            "start": 0.0,
            "end": dur,
            "label": "Noise",
            "filename": os.path.basename(nf)
        }
        noise_annotations.append(ann)
    except Exception as e:
        print(f"Error reading {nf}: {e}")

noise_json_path = os.path.join(JSON_DIR, "noise_annotations.json")
with open(noise_json_path, 'w') as f:
    json.dump(noise_annotations, f, indent=4)

# 3. Generate Spectrograms
print("Generating Spectrograms...")

# Combine bat audio dirs and noise audio dir into one list for the processor
ALL_AUDIO_DIRS = RAW_AUDIO
import os
import json
import glob
import librosa
import sys

# Ensure we are in the right directory for imports
if os.getcwd().split('/')[-1] != 'IndianBatsModel':
    if os.path.exists('IndianBatsModel'):
        os.chdir('IndianBatsModel')
    sys.path.append('.')

from MainShitz.data_prep.wombat_to_spectrograms import process_all as generate_spectrograms
from MainShitz.data_prep.whombat_project_to_wombat import convert_whombat_project_to_wombat_jsons

# --- CONFIGURATION ---

# 1. Input Paths (Adjust these to match your Kaggle Dataset structure)
#    These are the folders containing your .wav files
RAW_AUDIO_DIRS = [
    '/kaggle/input/annotations-tenuis-ceylonicus/Pip ceylonicus',
    '/kaggle/input/annotations-tenuis-ceylonicus/Pip._tenuis'
]

#    These are the JSON exports from Whombat
WHOMBAT_PROJECT_JSONS = [
    '/kaggle/input/annotations-tenuis-ceylonicus/tenuis annotations.json',
    '/kaggle/input/annotations-tenuis-ceylonicus/Pip ceylonicus.json',
]

#    Path to your noise data (UPDATE THIS if your folder name is different)
NOISE_AUDIO_DIR = '/kaggle/input/noice-files/Noise' 

# 2. Output Paths (In the writable /kaggle/working directory)
JSON_DIR = '/kaggle/working/data/annotations_json_folder'
SPECT_OUT = '/kaggle/working/data/processed/spectrograms'

os.makedirs(JSON_DIR, exist_ok=True)
os.makedirs(SPECT_OUT, exist_ok=True)

# --- EXECUTION ---

# 1. Convert Bat Annotations
print("Converting Bat Annotations...")
for pj in WHOMBAT_PROJECT_JSONS:
    # Fix: Only pass 2 arguments (Input File, Output Dir)
    convert_whombat_project_to_wombat_jsons(pj, JSON_DIR)

# 2. Generate Noise Annotations
print("Generating Noise Annotations...")
noise_files = glob.glob(os.path.join(NOISE_AUDIO_DIR, "*.wav"))
print(f"Found {len(noise_files)} noise files.")

noise_annotations = []
for nf in noise_files:
    try:
        # Handle different librosa versions for duration check
        try:
            dur = librosa.get_duration(path=nf)
        except TypeError:
            dur = librosa.get_duration(filename=nf)
            
        # Create a simple annotation for the whole file
        ann = {
            "start": 0.0,
            "end": dur,
            "label": "Noise",
            "filename": os.path.basename(nf)
        }
        noise_annotations.append(ann)
    except Exception as e:
        print(f"Error reading {nf}: {e}")

noise_json_path = os.path.join(JSON_DIR, "noise_annotations.json")
with open(noise_json_path, 'w') as f:
    json.dump(noise_annotations, f, indent=4)

# 3. Generate Spectrograms
print("Generating Spectrograms...")

# Combine bat audio dirs and noise audio dir into one list for the processor
ALL_AUDIO_DIRS = RAW_AUDIO_DIRS + [NOISE_AUDIO_DIR]

In [ ]:
# 6. Update Config and Run Tuner
import yaml
import torch
import os

# Check GPU availability
if torch.cuda.is_available():
    print(f"CUDA is available! Found {torch.cuda.device_count()} GPU(s).")
    for i in range(torch.cuda.device_count()):
        print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("WARNING: CUDA is NOT available. Training will run on CPU.")

# DEBUG: Verify Dataset Existence
print(f"Checking dataset at: {SPECT_OUT}")
if os.path.exists(SPECT_OUT):
    subdirs = [d for d in os.listdir(SPECT_OUT) if os.path.isdir(os.path.join(SPECT_OUT, d))]
    print(f"Found {len(subdirs)} class folders: {subdirs}")
    total_files = 0
    for d in subdirs:
        count = len(os.listdir(os.path.join(SPECT_OUT, d)))
        print(f"  - {d}: {count} images")
        total_files += count
    print(f"Total images found: {total_files}")
    
    if total_files == 0:
        print("CRITICAL ERROR: Dataset directory exists but is empty!")
else:
    print(f"CRITICAL ERROR: Dataset directory {SPECT_OUT} does not exist!")

config_path = 'configs/config.yaml'
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

# Update data path
config['data']['processed_data_path'] = SPECT_OUT
config['data']['train_spectrograms'] = SPECT_OUT

# Set epochs for tuning
config['train']['epochs'] = 5

# OPTIMIZATION: Increase workers to feed the GPU faster
config['train']['num_workers'] = 4 

with open(config_path, 'w') as f:
    yaml.dump(config, f)

print("Config updated. Starting Tuner...")

!python smart_tuner.py